In [ ]:
##!/usr/bin/env python
"""plot_skill_boxplots.py

Script does boxplots of the normalised error scores for WRF-LIS-CABLE physics ensemble

Author: Annette L Hirsch @ CLEX, UNSW. Sydney (Australia)
email: a.hirsch@unsw.edu.au
Created: Thu May  2 12:15:30 AEST 2019

"""

In [ ]:
# Load packages

#from __future__ import division
import numpy as np
import netCDF4 as nc
import sys
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray
from mpl_toolkits.basemap import Basemap
import common_functions as cf


In [ ]:
# Simulation Details - grouped by PBL

sims=["PHYS_TEST_RA1_PBL1_CU1", "PHYS_TEST_RA1_PBL1_CU16", "PHYS_TEST_RA1_PBL1_CU2", "PHYS_TEST_RA1_PBL1_CU3", 
      "PHYS_TEST_RA4_PBL1_CU1", "PHYS_TEST_RA4_PBL1_CU16", "PHYS_TEST_RA4_PBL1_CU2", "PHYS_TEST_RA4_PBL1_CU3", 
      "PHYS_TEST_RA5_PBL1_CU1", "PHYS_TEST_RA5_PBL1_CU16", "PHYS_TEST_RA5_PBL1_CU2", "PHYS_TEST_RA5_PBL1_CU3", 
      "PHYS_TEST_RA1_PBL2_CU1", "PHYS_TEST_RA1_PBL2_CU16", "PHYS_TEST_RA1_PBL2_CU2", "PHYS_TEST_RA1_PBL2_CU3", 
      "PHYS_TEST_RA3_PBL2_CU1", "PHYS_TEST_RA3_PBL2_CU16", "PHYS_TEST_RA3_PBL2_CU2", 
      "PHYS_TEST_RA4_PBL2_CU1", "PHYS_TEST_RA4_PBL2_CU16", "PHYS_TEST_RA4_PBL2_CU2", "PHYS_TEST_RA4_PBL2_CU3", 
      "PHYS_TEST_RA5_PBL2_CU1", "PHYS_TEST_RA5_PBL2_CU16", "PHYS_TEST_RA5_PBL2_CU2", "PHYS_TEST_RA5_PBL2_CU3", 
      "PHYS_TEST_RA1_PBL5_CU1", "PHYS_TEST_RA1_PBL5_CU16", "PHYS_TEST_RA1_PBL5_CU2", "PHYS_TEST_RA1_PBL5_CU3", 
      "PHYS_TEST_RA4_PBL5_CU1", "PHYS_TEST_RA4_PBL5_CU16", "PHYS_TEST_RA4_PBL5_CU2", "PHYS_TEST_RA4_PBL5_CU3", 
      "PHYS_TEST_RA5_PBL5_CU1", "PHYS_TEST_RA5_PBL5_CU16", "PHYS_TEST_RA5_PBL5_CU2", "PHYS_TEST_RA5_PBL5_CU3"]

PBL = [[0,1,2,3,4,5,6,7,8,9,10,11],[12,13,14,15,19,20,21,22,23,24,25,26],[27,28,29,30,31,32,33,34,35,36,37,38]]
PBLlbs = [['YSU'],['MYJ'],['MYNN']]
RA = [[0,1,2,3,12,13,14,15,27,28,29,30],[4,5,6,7,19,20,21,22,31,32,33,34],[8,9,10,11,23,24,25,26,35,36,37,38]]
RAlbs = [['Dudhia'],['RRTMG'],['NGoddard']]
CU = [[0,4,8,12,19,23,27,31,35],[1,5,9,13,20,24,28,32,36],[2,6,10,14,21,25,29,33,37],[3,7,11,15,22,26,30,34,38]]
CUlbs = [['KF'],['NTiedtke'],['BMJ'],['GF']]

PHYS = [[0,1,2,3,4,5,6,7,8,9,10,11],[12,13,14,15,19,20,21,22,23,24,25,26],[27,28,29,30,31,32,33,34,35,36,37,38],[0,1,2,3,12,13,14,15,27,28,29,30],[4,5,6,7,19,20,21,22,31,32,33,34],[8,9,10,11,23,24,25,26,35,36,37,38],[0,4,8,12,19,23,27,31,35],[1,5,9,13,20,24,28,32,36],[2,6,10,14,21,25,29,33,37],[3,7,11,15,22,26,30,34,38]]
PHYSlbs = [['YSU'],['MYJ'],['MYNN'],['Dudhia'],['RRTMG'],['NGoddard'],['KF'],['NTiedtke'],['BMJ'],['GF']]


In [ ]:
# Define USER input arguments

# Year / Period of interest
syear = np.int('2008')
eyear = np.int('2010')

# Data directory 
datadir = '/g/data/hh5/tmp/WRF-CABLE/AUS44/postprocess/'

nmod = len(sims)
obs = "AWAP"

# Figure Details
fig_dir='%s/figures/' %(os.getcwd())
fig_name_prefix='BOX_AWAP_MASKED'
if not os.path.exists(fig_dir):
  os.makedirs(fig_dir)

# Landsea mask
mask_file='/g/data/hh5/tmp/WRF-CABLE/AUS44/PHYS_TEST_MASTER/geo_em.d01.nc'
lis_file='/g/data/hh5/tmp/WRF-CABLE/AUS44/PHYS_TEST_MASTER/bdy_data/lis_input.d01.nc'

# Variables
variables=['PR','TX','TN','QE']
nvar=len(variables)
mxmn = [3,10,10,50.]
vlabels=['PR [mm $day{^-1}$]','TX [\xb0 C]','TN [\xb0 C]','$Q_{E}$ [W $m{^-2}$]']

# Define regions to zoom in on
#latN=[-43.0,-23.43,-38.0,-34.0]
#latX=[-11.0,-10.0,-27.0,-30.0]
#lonN=[116.0,110.0,139.0,114.0]
#lonX=[155.0,155.0,151.0,120.0]
#Rname=["Australia","TROPICS","SEA","SWWA"]
#nreg = len(Rname)

latN=[-23.43,-38.0,-34.0]
latX=[-10.0,-27.0,-30.0]
lonN=[110.0,139.0,114.0]
lonX=[155.0,151.0,120.0]
Rname=["TROPICS","SEA","SWWA"]
nreg = len(Rname)

# Define the mean climate intervals of interest    
mclim=["ANN","DJF","MAM","JJA","SON"]
ytarget=["2009"]
mtarget=[["12","01","02"],["03","04","05"],["06","07","08"],["09","10","11"]]
nmean = len(mclim)


In [ ]:
# Load the mask file
filemask    = nc.Dataset(lis_file)
lsmask      = filemask.variables['LANDMASK'][:]
lsmask      = lsmask.astype(int)  
lat2d       = filemask.variables['lat'][:]
lon2d       = filemask.variables['lon'][:]
filemask.close()

In [ ]:
# Function to create plot
def plot_boxplot(data,pind,plabels,figurename,mlabel,mx=1.,mn=0.):
    """This function plots model skill in the context of observational uncertainty.
    
    Input arguments: 
        data - multi-dimensional array containing the skill scores
        RCMlabels - list containing model/experiment names
        figurename - file name for saving the figure to
        vlabels - array containing the variables included
        Rname - region names
        mtitle - main title
        mx = max value on colorbar
        mn = min value on colorbar
    """

    from matplotlib.colors import BoundaryNorm
    from matplotlib.ticker import MaxNLocator
    from matplotlib.colors import LinearSegmentedColormap

    # Retrieve dimensions
    nmod = data.shape[0] # Models
    nphys = len(pind) # Number of physics schemes
        
    # Create figure object and subplots
    plt.rcParams['savefig.dpi'] = 500
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"
    
    mycolors = ["black","black","black","black"]
    
    # Create figure object and subplots
    if nphys == 10:
        fig, (ax1, ax2, ax3, ax4,ax5,ax6,ax7,ax8,ax9,ax10) = plt.subplots(1, nphys, figsize=(7.,6.),sharex=False, sharey=False)

    elif nphys == 4:
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, nphys, figsize=(2.0*(nphys),6.),sharex=False, sharey=False)
    else:
        fig, (ax1, ax2, ax3) = plt.subplots(1, nphys, figsize=(2.0*(nphys),6.), sharex=False, sharey=False)

    # Filter the data to remove the nans
    data1 = np.reshape(data[pind[0],:,:,:],-1)
    filtered_data1 = data1[~np.isnan(data1)]
    data2 = np.reshape(data[pind[1],:,:,:],-1)
    filtered_data2 = data2[~np.isnan(data2)]
    data3 = np.reshape(data[pind[2],:,:,:],-1)
    filtered_data3 = data3[~np.isnan(data3)]
    if nphys == 4:
        data4 = np.reshape(data[pind[3],:,:,:],-1)
        filtered_data4 = data4[~np.isnan(data4)]
    if nphys == 10:
        data4 = np.reshape(data[pind[3],:,:,:],-1)
        filtered_data4 = data4[~np.isnan(data4)]
        data5 = np.reshape(data[pind[4],:,:,:],-1)
        filtered_data5 = data5[~np.isnan(data5)]
        data6 = np.reshape(data[pind[5],:,:,:],-1)
        filtered_data6 = data6[~np.isnan(data6)]
        data7 = np.reshape(data[pind[6],:,:,:],-1)
        filtered_data7 = data7[~np.isnan(data7)]
        data8 = np.reshape(data[pind[7],:,:,:],-1)
        filtered_data8 = data8[~np.isnan(data8)]
        data9 = np.reshape(data[pind[8],:,:,:],-1)
        filtered_data9 = data9[~np.isnan(data9)]
        data10 = np.reshape(data[pind[9],:,:,:],-1)
        filtered_data10 = data10[~np.isnan(data10)]
    
    # Plot data as a box plot
    bp1 = ax1.boxplot(filtered_data1, patch_artist=False, showfliers=False, widths = 0.75)
    ax1.set_xticklabels(plabels[0],rotation=90.,fontsize=18)
    ax1.set_ylim(mn,mx)
#    ax1.set_yticks(np.arange(mn,mx))
    ax1.set_yticklabels(np.arange(mn,mx), fontsize=18)
    bp2 = ax2.boxplot(filtered_data2, patch_artist=False, showfliers=False, widths = 0.75)
    ax2.set_xticklabels(plabels[1],rotation=90.,fontsize=18)
    ax2.set_ylim(mn,mx)
    ax2.set_yticks([],[])
    bp3 = ax3.boxplot(filtered_data3, patch_artist=False, showfliers=False, widths = 0.75)
    ax3.set_xticklabels(plabels[2],rotation=90.,fontsize=18)
    ax3.set_ylim(mn,mx)
    ax3.set_yticks([],[])
    
    ## change outline color, fill color and linewidth of the boxes
    for box in bp1['boxes']:
        # change outline color
        box.set( color='black', linewidth=2)
    ## change color and linewidth of the whiskers
    for whisker in bp1['whiskers']:
        whisker.set(color='black', linewidth=2)
    ## change color and linewidth of the caps
    for cap in bp1['caps']:
        cap.set(color='black', linewidth=2)
    ## change color and linewidth of the medians
    for median in bp1['medians']:
        median.set(color='red', linewidth=3)
    
    for box in bp2['boxes']:
        box.set( color='black', linewidth=2)
    for whisker in bp2['whiskers']:
        whisker.set(color='black', linewidth=2)
    for cap in bp2['caps']:
        cap.set(color='black', linewidth=2)
    for median in bp2['medians']:
        median.set(color='red', linewidth=3)

    for box in bp3['boxes']:
        box.set( color='black', linewidth=2)
    for whisker in bp3['whiskers']:
        whisker.set(color='black', linewidth=2)
    for cap in bp3['caps']:
        cap.set(color='black', linewidth=2)
    for median in bp3['medians']:
        median.set(color='red', linewidth=3)
        
    if nphys == 4:
        bp4 = ax4.boxplot(filtered_data4, patch_artist=False, showfliers=False, widths = 0.75)
        ax4.set_xticklabels(plabels[3],rotation=90.,fontsize=18)
        ax4.set_ylim(mn,mx)
        ax4.set_yticks([],[])
        ax4.axhline(0., color='grey', linestyle='--',linewidth=1.0)

        for box in bp4['boxes']:
            box.set( color='black', linewidth=2)
        for whisker in bp4['whiskers']:
            whisker.set(color='black', linewidth=2)
        for cap in bp4['caps']:
            cap.set(color='black', linewidth=2)
        for median in bp4['medians']:
            median.set(color='red', linewidth=3)

    if nphys == 10:
        bp4 = ax4.boxplot(filtered_data4, patch_artist=False, showfliers=False, widths = 0.75)
        ax4.set_xticklabels(plabels[3],rotation=90.,fontsize=18)
        ax4.set_ylim(mn,mx)
        ax4.set_yticks([],[])
        ax4.axhline(0., color='grey', linestyle='--',linewidth=1.0)
        for box in bp4['boxes']:
            box.set( color='black', linewidth=2)
        for whisker in bp4['whiskers']:
            whisker.set(color='black', linewidth=2)
        for cap in bp4['caps']:
            cap.set(color='black', linewidth=2)
        for median in bp4['medians']:
            median.set(color='red', linewidth=3)
            
        bp5 = ax5.boxplot(filtered_data5, patch_artist=False, showfliers=False, widths = 0.75)
        ax5.set_xticklabels(plabels[4],rotation=90.,fontsize=18)
        ax5.set_ylim(mn,mx)
        ax5.set_yticks([],[])
        ax5.axhline(0., color='grey', linestyle='--',linewidth=1.0)
        for box in bp5['boxes']:
            box.set( color='black', linewidth=2)
        for whisker in bp5['whiskers']:
            whisker.set(color='black', linewidth=2)
        for cap in bp5['caps']:
            cap.set(color='black', linewidth=2)
        for median in bp5['medians']:
            median.set(color='red', linewidth=3)
            
        bp6 = ax6.boxplot(filtered_data6, patch_artist=False, showfliers=False, widths = 0.75)
        ax6.set_xticklabels(plabels[5],rotation=90.,fontsize=18)
        ax6.set_ylim(mn,mx)
        ax6.set_yticks([],[])
        ax6.axhline(0., color='grey', linestyle='--',linewidth=1.0)
        for box in bp6['boxes']:
            box.set( color='black', linewidth=2)
        for whisker in bp6['whiskers']:
            whisker.set(color='black', linewidth=2)
        for cap in bp6['caps']:
            cap.set(color='black', linewidth=2)
        for median in bp6['medians']:
            median.set(color='red', linewidth=3)
            
        bp7 = ax7.boxplot(filtered_data7, patch_artist=False, showfliers=False, widths = 0.75)
        ax7.set_xticklabels(plabels[6],rotation=90.,fontsize=18)
        ax7.set_ylim(mn,mx)
        ax7.set_yticks([],[])
        ax7.axhline(0., color='grey', linestyle='--',linewidth=1.0)
        for box in bp7['boxes']:
            box.set( color='black', linewidth=2)
        for whisker in bp7['whiskers']:
            whisker.set(color='black', linewidth=2)
        for cap in bp7['caps']:
            cap.set(color='black', linewidth=2)
        for median in bp7['medians']:
            median.set(color='red', linewidth=3)
            
        bp8 = ax8.boxplot(filtered_data8, patch_artist=False, showfliers=False, widths = 0.75)
        ax8.set_xticklabels(plabels[7],rotation=90.,fontsize=18)
        ax8.set_ylim(mn,mx)
        ax8.set_yticks([],[])
        ax8.axhline(0., color='grey', linestyle='--',linewidth=1.0)
        for box in bp8['boxes']:
            box.set( color='black', linewidth=2)
        for whisker in bp8['whiskers']:
            whisker.set(color='black', linewidth=2)
        for cap in bp8['caps']:
            cap.set(color='black', linewidth=2)
        for median in bp8['medians']:
            median.set(color='red', linewidth=3)
            
        bp9 = ax9.boxplot(filtered_data9, patch_artist=False, showfliers=False, widths = 0.75)
        ax9.set_xticklabels(plabels[8],rotation=90.,fontsize=18)
        ax9.set_ylim(mn,mx)
        ax9.set_yticks([],[])
        ax9.axhline(0., color='grey', linestyle='--',linewidth=1.0)
        for box in bp9['boxes']:
            box.set( color='black', linewidth=2)
        for whisker in bp9['whiskers']:
            whisker.set(color='black', linewidth=2)
        for cap in bp9['caps']:
            cap.set(color='black', linewidth=2)
        for median in bp9['medians']:
            median.set(color='red', linewidth=3)
            
        bp10 = ax10.boxplot(filtered_data10, patch_artist=False, showfliers=False, widths = 0.75)
        ax10.set_xticklabels(plabels[9],rotation=90.,fontsize=18)
        ax10.set_ylim(mn,mx)
        ax10.set_yticks([],[])
        ax10.axhline(0., color='grey', linestyle='--',linewidth=1.0)
        for box in bp10['boxes']:
            box.set( color='black', linewidth=2)
        for whisker in bp10['whiskers']:
            whisker.set(color='black', linewidth=2)
        for cap in bp10['caps']:
            cap.set(color='black', linewidth=2)
        for median in bp10['medians']:
            median.set(color='red', linewidth=3)
            
    ax1.axhline(0., color='grey', linestyle='--',linewidth=1.0)
    ax2.axhline(0., color='grey', linestyle='--',linewidth=1.0)
    ax3.axhline(0., color='grey', linestyle='--',linewidth=1.0)

    # Amend the x=y-axis labels
    ax1.set_ylabel('Model Bias %s' %(mlabel))
        
    # Finalise the figure layout
    
    fig.tight_layout()   
    fig.subplots_adjust(wspace=0, hspace=0, left=0.25)

    fig.savefig(figurename,bbox_inches = "tight")
    plt.close(fig)


In [ ]:
# Load observational data

filename='%s/%s_regrid_daily_%s_%s_mask.nc' %(datadir,obs,syear,eyear)
file = nc.Dataset(filename)
obsprecip = file.variables['awap_precip_mask'][:]
obst2max = file.variables['awap_t2max_mask'][:]
obst2min = file.variables['awap_t2min_mask'][:]
file.close()

filename='%s/GLEAM_regrid_daily_%s_%s.nc' %(datadir,syear,eyear)
file = nc.Dataset(filename)
obshfls = file.variables['gleam_hfls_mask'][:]
file.close()

ndays = 822-61
sind = 213+61

# Truncate data to simulation period - removing the spinup aswell
obsprecipT = obsprecip[sind:sind+ndays,:,:]
obst2maxT = obst2max[sind:sind+ndays,:,:]
obst2minT = obst2min[sind:sind+ndays,:,:]
obshflsT = obshfls[sind:sind+ndays,:,:]

# Mask missing values
obsprecipma = np.ma.masked_array(obsprecipT, obsprecipT>=1.e20).filled(np.nan)
obst2maxma  = np.ma.masked_array(obst2maxT, obst2maxT>=1.e20).filled(np.nan)
obst2minma  = np.ma.masked_array(obst2minT, obst2minT>=1.e20).filled(np.nan)
obshflsma  = np.ma.masked_array(obshflsT, obst2maxT>=1.e20).filled(np.nan)


In [ ]:

# Loop through regions

for rind,rname in enumerate(Rname):

    # Define region
    bbox = [lonN[rind],lonX[rind], latN[rind], latX[rind]]
    i0,i1,j0,j1 = cf.bbox2ij(lon2d,lat2d,bbox)
    ny = lat2d[j0:j1,i0:i1].shape[0]
    nx = lat2d[j0:j1,i0:i1].shape[1]
    bias = np.empty((nmod,nmean,nvar,ny,nx),dtype=np.float64)

    # Loop through the experiments
    for mind,mname in enumerate(sims):

        # Load precip data - excluding 2 month spinup
        filename='%s/%s_WRF_Precip_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
        file = nc.Dataset(filename)
        precip = file.variables['wrf_precip_mask'][61:,:,:]
        file.close()

        # Load temperature data - excluding 2 month spinup
        filename='%s/%s_WRFext_Temps_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
        file = nc.Dataset(filename)
        t2max = file.variables['T2MAX'][61:,:,:]
        t2min = file.variables['T2MIN'][61:,:,:]
        time = nc.chartostring(file.variables['Times'][61:,:])
        timesplit = np.asarray([time[x].split("_")[0] for x in range(len(time))])
        years = np.asarray([timesplit[x].split("-")[0] for x in range(len(time))])
        months = np.asarray([timesplit[x].split("-")[1] for x in range(len(time))])
        file.close()

        # Load hfls data - excluding 2 month spinup
        filename='%s/%s_lisout_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
        file = nc.Dataset(filename)
        hfls = file.variables['lis_hfls_mask'][61:,:,:]
        lat2d = file.variables['wrf_lat2d'][:]
        lon2d = file.variables['wrf_lon2d'][:]
        file.close()

        # Truncate to the period of interest for climate means
        for pind,pname in enumerate(mclim):

            # Get relevant time indices
            if pname in ["ANN"]:
                tind = np.where(years==ytarget)[0]
            else:
                tind = np.asarray([months[x] in mtarget[pind-1] for x in range(len(months))])

            # Calculate the bias score
            bias[mind,pind,0,:,:] = np.nanmean(precip[tind,j0:j1,i0:i1],axis=0) - np.nanmean(obsprecipma[tind,j0:j1,i0:i1],axis=0)
            bias[mind,pind,1,:,:] = np.nanmean(t2max[tind,j0:j1,i0:i1],axis=0) - np.nanmean(obst2maxma[tind,j0:j1,i0:i1],axis=0)
            bias[mind,pind,2,:,:] = np.nanmean(t2min[tind,j0:j1,i0:i1],axis=0) - np.nanmean(obst2minma[tind,j0:j1,i0:i1],axis=0)
            bias[mind,pind,3,:,:] = np.nanmean(hfls[tind,j0:j1,i0:i1],axis=0) - np.nanmean(obshflsma[tind,j0:j1,i0:i1],axis=0)

            del tind

        del precip, t2max, t2min, hfls
        del time,timesplit,years,months

    for vind in range(nvar):
        figurename='%s/%s_%s_%s.png' %(fig_dir,'BOX_NO_CAM_AWAP_MASKED',variables[vind],rname)
        plot_boxplot(bias[:,:,vind,:,:],PHYS,PHYSlbs,figurename,vlabels[vind],mx=mxmn[vind],mn=-mxmn[vind])
        
    del bias,bbox,i0,i1,j0,j1
